# 지니 Top200 차트

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

### 1. 인터넷에서 데이터 가져오기

In [2]:
url = 'https://www.genie.co.kr/chart/top200'
req = requests.get(url)
req.text

'<br>\r\n<br>\r\n<center>\r\n<img src="http://www.geniemusic.co.kr/images/common/logo_r1.png"><br>\r\n<h2> <meta http-equiv="Content-Type" content="text/html;charset=UTF-8"> ì\xa0\x91ì\x86\x8dì\x9a\x94ì²\xadì\x9d´ ë³´ì\x95\x88ì\xa0\x95ì±\x85ì\x97\x90 ì\x9d\x98í\x95´ ì°¨ë\x8b¨ë\x90\x98ì\x97\x88ì\x8aµë\x8b\x88ë\x8b¤. ë\x8b¹ì\x82¬ ê³\xa0ê°\x9dì\x84¼í\x84°ë¡\x9c ë¬¸ì\x9d\x98í\x95´ì£¼ì\x8b\xadì\x8b\x9cì\x98¤.<br><br>\r\nThe security policy of the connection request is blocked. Contact your customer service representative.<br><br>\r\nì§\x80ë\x8b\x88ë®¤ì§\x81 ê³\xa0ê°\x9dì\x84¼í\x84° 1577-5337<br><br>\r\n</h2>\r\n</center>\r\n<br>'

In [3]:
# Chrome User-Agent
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.88 Safari/537.36'}
req = requests.get(url, headers=header)
soup = BeautifulSoup(req.text)

### 2. 찾으려고 하는 데이터의 노드 찾기

In [4]:
trs = soup.select('tr.list')
len(trs)

50

In [5]:
# 개발자 도구에서 찾아주는 Selector
trs = soup.select('#body-content > div.newest-list > div > table > tbody > tr')
len(trs)

50

### 3. 여러개의 데이터 중 하나를 선택해서 원하는 정보를 추출

In [24]:
tr = trs[12]

In [25]:
# rank
tr.select_one('.number').get_text()

'13\n                                        \n                                    \n2상승\n\n'

In [26]:
try:
    rank = int(tr.select_one('.number').get_text().split('\n')[0])
except:
    rank = 0
rank

13

In [32]:
# 제목
try:            # 일반적인 노래
    title = tr.select_one('.title.ellipsis').string.strip()
except:         # 19금 노래
    title = tr.select_one('.title.ellipsis').get_text().split('\n')[-1].strip()
title

'abcdefu'

In [17]:
# artist
artist = tr.select_one('.artist.ellipsis').text.strip()
artist

'BIGBANG (빅뱅)'

In [20]:
# album
album = tr.select_one('.albumtitle.ellipsis').text.strip()
album

'봄여름가을겨울 (Still Life)'

### 4. 한 페이지내에 있는 정보

- 1) 개별 리스트로 가져오기

In [23]:
# 어디에서 오류가 발생하는지 확인
rank_list, title_list, artist_list, album_list = [],[],[],[]
for i, tr in enumerate(trs):
    try:
        rank = int(tr.select_one('.number').get_text().split('\n')[0])
    except:
        rank = 0
    try:
        title = tr.select_one('.title.ellipsis').string.strip()
        artist = tr.select_one('.artist.ellipsis').text.strip()
        album = tr.select_one('.albumtitle.ellipsis').text.strip()
    except:
        print(i)
    rank_list.append(rank)
    title_list.append(title)
    artist_list.append(artist)
    album_list.append(album)

12


In [33]:
rank_list, title_list, artist_list, album_list = [],[],[],[]
for i, tr in enumerate(trs):
    try:
        rank = int(tr.select_one('.number').get_text().split('\n')[0])
    except:
        rank = 0
    try:            # 일반적인 노래
        title = tr.select_one('.title.ellipsis').string.strip()
    except:         # 19금 노래
        title = tr.select_one('.title.ellipsis').get_text().split('\n')[-1].strip()
    artist = tr.select_one('.artist.ellipsis').text.strip()
    album = tr.select_one('.albumtitle.ellipsis').text.strip()
    rank_list.append(rank)
    title_list.append(title)
    artist_list.append(artist)
    album_list.append(album)

- 2) 줄단위로 딕셔너리 리스트를 만들기

In [37]:
lines = []
for i, tr in enumerate(trs):
    try:
        rank = int(tr.select_one('.number').get_text().split('\n')[0])
    except:
        rank = 0
    try:            # 일반적인 노래
        title = tr.select_one('.title.ellipsis').string.strip()
    except:         # 19금 노래
        title = tr.select_one('.title.ellipsis').get_text().split('\n')[-1].strip()
    artist = tr.select_one('.artist.ellipsis').text.strip()
    album = tr.select_one('.albumtitle.ellipsis').text.strip()
    line = {'순위':rank, '제목':title, '아티스트':artist, '앨범':album}
    lines.append(line)

### 5. 데이터 프레임으로 만들기

In [35]:
df = pd.DataFrame({'순위':rank_list, '제목':title_list,
                '아티스트':artist_list, '앨범':album_list})
df.head(15).tail(5)

,순위,제목,아티스트,앨범
10,11,LOVE me,BE'O (비오),LOVE me
11,12,다정히 내 이름을 부르면,경서예지 & 전건호,다정히 내 이름을 부르면 (경서예지 x 전건호)
12,13,abcdefu,GAYLE,abcdefu
13,14,신호등,이무진,신호등
14,15,ELEVEN,IVE (아이브),ELEVEN


In [38]:
df = pd.DataFrame(lines)
df.head(15).tail(5)

,순위,제목,아티스트,앨범
10,11,LOVE me,BE'O (비오),LOVE me
11,12,다정히 내 이름을 부르면,경서예지 & 전건호,다정히 내 이름을 부르면 (경서예지 x 전건호)
12,13,abcdefu,GAYLE,abcdefu
13,14,신호등,이무진,신호등
14,15,ELEVEN,IVE (아이브),ELEVEN


### 6. 모든 페이지에 대해 가져오기

In [39]:
lines = []
for page in range(1,5):
    url = f'https://www.genie.co.kr/chart/top200?ditc=D&ymd=20220415&hh=10&rtm=Y&pg={page}'
    req = requests.get(url, headers=header)
    soup = BeautifulSoup(req.text, 'html.parser')
    trs = soup.select('tr.list')

    for tr in trs:
        try:
            rank = int(tr.select_one('.number').get_text().split('\n')[0])
        except:
            rank = 0
        try:            # 일반적인 노래
            title = tr.select_one('.title.ellipsis').string.strip()
        except:         # 19금 노래
            title = tr.select_one('.title.ellipsis').get_text().split('\n')[-1].strip()
        artist = tr.select_one('.artist.ellipsis').text.strip()
        album = tr.select_one('.albumtitle.ellipsis').text.strip()
        line = {'순위':rank, '제목':title, '아티스트':artist, '앨범':album}
        lines.append(line)

In [40]:
df = pd.DataFrame(lines)
df.tail()

,순위,제목,아티스트,앨범
195,196,하루하루,BIGBANG (빅뱅),Stand Up (2008 빅뱅 3rd Mini Album)
196,197,붉은 노을,BIGBANG (빅뱅),REMEMBER
197,198,Saturday Drip,NCT DREAM,Glitch Mode - The 2nd Album
198,199,"너, 너",스트레이 (The Stray),her
199,200,어른,Sondia,나의 아저씨 OST Part. 2 (tvN 수목드라마)


In [41]:
df.to_csv('지니차트Top200.csv', index=False)